In [0]:
import nltk
import pandas as pd
import numpy as np

In [0]:
!pip install pymorphy2
!nltk.download('stopwords')
!nltk.download('words')
!nltk.download('averaged_perceptron_tagger')
!nltk.download('wordnet')
!nltk.download('punkt')

In [0]:
import re
from nltk.stem import WordNetLemmatizer
from string import punctuation
from nltk.tokenize import word_tokenize
from nltk.corpus import words
from nltk import pos_tag
from nltk.stem import PorterStemmer 

lemmatizer = WordNetLemmatizer() 

Загружаем подготовленный датасет спаршенных английских сайтов

In [0]:
alexaWC = pd.read_csv('/content/drive/My Drive/Colab Notebooks/AlexaFull.csv', encoding='utf-8-sig', lineterminator='\n')
alexaWC = alexaWC.drop('Unnamed: 0',1)
alexaWC = alexaWC.fillna('')

In [0]:
stop_wordsEN = nltk.corpus.stopwords.words("english")
stop_wordsEN = set(stop_wordsEN)
allWords = set(words.words())

Создаем функцию, которая будет предобрабатывать английский текст <br>
Функция убирает все не англйские символы, лемматизирует слова, переводя их в начальную форму и отбрасывает все несуществующие слова <br>
Возвращает предобработанный текст

In [0]:
def textPreprocess(text):

  result = re.sub(r'[^a-zA-Z ]', '', text)
  array = []
  tokens = word_tokenize(result.lower())
  
  if len(tokens)!=0:
    for i,j in pos_tag(tokens):
      if j[0].lower() in ['a','n','v']:
        lemma = lemmatizer.lemmatize(i,j[0].lower())
      else:
        lemma = lemmatizer.lemmatize(i)
      
      if lemma not in stop_wordsEN and lemma in allWords and lemma != ' ':
        array.append(lemma)
  
  textEN = ' '.join(array

  return textEN

Прогоняем созданную ранее функцию через весь загруженный датасет спаршенных английских сайтов, используя мультипроцессинг для ускорения работы

In [0]:
content = alexaWC['content'].to_list()
titles = alexaWC['title'].to_list()
descriptns = alexaWC['description'].to_list()

content_length = len(content)

In [0]:
with concurrent.futures.ProcessPoolExecutor() as executor:
    r = list(tqdm.tqdm(executor.map(delMostWords, content), total=content_length))

alexaWC['content'] = r

with concurrent.futures.ProcessPoolExecutor() as executor:
    r = list(tqdm.tqdm(executor.map(delMostWords, titles), total=content_length))

alexaWC['title'] = r

with concurrent.futures.ProcessPoolExecutor() as executor:
    r = list(tqdm.tqdm(executor.map(delMostWords, descriptns), total=content_length))

alexaWC['description'] = r

In [0]:
alexaWC.to_csv('/content/drive/My Drive/Colab Notebooks/AlexaFullLemma.csv', encoding='utf-8-sig')